# Battle of Neighborhoods Coding Notebook. 
## OPENING A NEW RESTAURANT IN SAN FRANCISCO

### [ INTRODUCTION ]
### Our company wants to open a new restaurant in San Francisco. The interest is to open one at a premium location within the city, that would bring good revenue for the new business. The decision is to rent the property instead of buying it.

In [2]:
#Importing all required packages.
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 



import folium
import pandas as pd
import requests
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

### Importing of the rent Dataset from my GitHub account to the Notebook.

In [5]:
git = 'https://raw.githubusercontent.com/Auxilin/Battle-of-Neighborhoods-for-new-Restaurant-location/master/SFRent_Dataset.csv'
sfran_Data = pd.read_csv(git)
sfran_Data

,Neighborhood,City,State,2018-07
0,Hayes Valley,San Francisco,CA,3030
1,Van Ness - Civic Center,San Francisco,CA,2977
2,Tenderloin,San Francisco,CA,2977
3,Downtown,San Francisco,CA,3040
4,Western Addition,San Francisco,CA,2989
5,Marina,San Francisco,CA,2948
6,Russian Hill,San Francisco,CA,2954
7,Lower Pacific Heights,San Francisco,CA,2963
8,Nob Hill,San Francisco,CA,3050
9,Pacific Heights,San Francisco,CA,2973


### For further analysis we need the co-ordinates of each neighborhood.

In [7]:
#creating a dataframe for storing co-ordinates details.
coordinates = pd.DataFrame(columns=['Latitude','Longitude'])

# Using 'for loop' to get pass each Neighborhood name and get co-ordinates details through geocoding.
for row,neighborhood in sfran_Data.iterrows():
    address = neighborhood['Neighborhood'] + ',' + neighborhood['City'] + ',' + neighborhood['State'] 
    try:
        geolocator = Nominatim(user_agent="my-application")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        # appending latitude and longitude values on coordiantes dataframe.
        coordinates = coordinates.append({'Latitude':latitude,'Longitude':longitude},ignore_index=True)
    except:
        print(address)
    

coordinates

Merced Heights,San Francisco,CA


,Latitude,Longitude
0,37.776685,-122.422936
1,37.775190,-122.419266
2,37.784249,-122.413993
3,37.787514,-122.407159
4,37.779559,-122.429810
5,37.799793,-122.435205
6,37.797707,-122.414971
7,37.785767,-122.438904
8,37.793262,-122.415249
9,37.792717,-122.435644


### Resize our sfran_Data dataframe to only Neighborhood & Rent information, then add both with co-ordinates.

In [8]:
#Let's now take only Neighorhood and rent data for further testing.
sfran_Data= sfran_Data[['Neighborhood','2018-07']]

# Adding the sfran_Data and coordinates in one new dataframe.
sf_Neighborhood = sfran_Data.join(coordinates, how='outer')
sf_Neighborhood

,Neighborhood,2018-07,Latitude,Longitude
0,Hayes Valley,3030,37.776685,-122.422936
1,Van Ness - Civic Center,2977,37.775190,-122.419266
2,Tenderloin,2977,37.784249,-122.413993
3,Downtown,3040,37.787514,-122.407159
4,Western Addition,2989,37.779559,-122.429810
5,Marina,2948,37.799793,-122.435205
6,Russian Hill,2954,37.797707,-122.414971
7,Lower Pacific Heights,2963,37.785767,-122.438904
8,Nob Hill,3050,37.793262,-122.415249
9,Pacific Heights,2973,37.792717,-122.435644


### Now let's calculate rentScore and RatingScore from our formaula i.e finalScore = (rentScore)0.6+(ratingScore)0.4
### First to calculate the rentSCore formula is (maxrentofN-currentrentofN)/(maxrentofN-minrentofN). Maximum and minimum values can be retrieved using max and min built in functions.

In [9]:
# Max value in '2018-07'
maxrentofN = max(sf_Neighborhood['2018-07'])
print('Max rent value of SF Neighborhood is $',maxrentofN)
minrentofN = min(sf_Neighborhood['2018-07'])
print('Min rent value SF Neighborhood is $',minrentofN)

Max rent value of SF Neighborhood is $ 3711
Min rent value SF Neighborhood is $ 2640


In [14]:
#create a new dataframe to store rent score.
rent_Score = pd.DataFrame(columns=['RentScore'])
for index, Neighborhood in sf_Neighborhood.iterrows():
    currentrentofN = Neighborhood['2018-07']
    rentscore =  (maxrentofN-currentrentofN)/(maxrentofN-minrentofN)
    rent_Score = rent_Score.append({'RentScore' : rentscore},ignore_index = True)
    
rent_Score 

,RentScore
0,0.635854
1,0.685341
2,0.685341
3,0.626517
4,0.674136
5,0.712418
6,0.706816
7,0.698413
8,0.617180
9,0.689076


In [16]:
#adding rent score information in sf_Neighborhood dataframe. 
sf_Neighborhood = sf_Neighborhood.join(rent_Score, how='outer')
sf_Neighborhood

ValueError: columns overlap but no suffix specified: Index(['RentScore'], dtype='object')

### Now let's calculate the second part of the formula i.e ratingScore, In our project we need only relevent restaurant data, so let's use categoryid in search endpoint url.

In [17]:
# Restaurant categoryid of foursquare
categoryId= '4bf58dd8d48988d10f941735'
# url willl search in 500 meters radius of latitude and longitude.
radius=500
api_endpoint = 'https://api.foursquare.com/'
#used below url to cache the request made to foursquare api
api_endpoint = 'http://cladiusfernando-eval-test.apigee.net/foursquare/'

#dataframe to save venue information.
venue_Details = pd.DataFrame(columns=['VenueNeighborhoodName','VenueName','VenueRating'])
#dataframe to save count of good rating information.
goodRating =pd.DataFrame(columns=['Neighborhood','GoodRatingRestaurant'])

# 'for loop' to get venue information search endpoint
for row,neighborhood in sf_Neighborhood.iterrows():
    venue_NeighborhoodName = neighborhood['Neighborhood']
    lat = neighborhood['Latitude']
    long = neighborhood['Longitude']
    url = '{}v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}'.format(
    api_endpoint,
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius,
    categoryId)
    
    venues = requests.get(url).json()['response']['venues']
    goodrating = 0
    for venue in venues:
        venuen = venue['name']         #retrieving venue name neighborhood-wise
        VENUE_ID = venue['id']         #retrieving venue id neighborhood-wise  
        url2 = '{}v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        api_endpoint,
        VENUE_ID,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION
        )
        ven = requests.get(url2).json()['response']['venue']
        
         #if statement to get rating values for each venue.
        rating = 0
        if 'rating' in ven:
            rating = ven['rating']
            venue_Details = venue_Details.append({'VenueNeighborhoodName': venue_NeighborhoodName,'VenueName' : venuen, 'VenueRating': rating}, ignore_index = True)
       
            #if statement to get good restaurant count neighborhood-wise.
            if rating >= 7.0:
                goodrating = goodrating+1
    #appending values in goodrating dataframe          
    goodRating =goodRating.append({'Neighborhood': venue_NeighborhoodName , 'GoodRatingRestaurant' : goodrating}, ignore_index = True)

goodRating


NameError: name 'CLIENT_ID' is not defined

### Below are the venue names and it's rating according to each neighborhood

In [18]:
venue_Details

,VenueNeighborhoodName,VenueName,VenueRating


In [19]:
# Let's join/add good rating count restaurants on sf_Neighborhood dataframe.
sf_Neighborhood = sf_Neighborhood.merge(goodRating, how='outer')
sf_Neighborhood

,Neighborhood,2018-07,Latitude,Longitude,RentScore,GoodRatingRestaurant
0,Hayes Valley,3030,37.776685,-122.422936,0.635854,NaN
1,Van Ness - Civic Center,2977,37.775190,-122.419266,0.685341,NaN
2,Tenderloin,2977,37.784249,-122.413993,0.685341,NaN
3,Downtown,3040,37.787514,-122.407159,0.626517,NaN
4,Western Addition,2989,37.779559,-122.429810,0.674136,NaN
5,Marina,2948,37.799793,-122.435205,0.712418,NaN
6,Russian Hill,2954,37.797707,-122.414971,0.706816,NaN
7,Lower Pacific Heights,2963,37.785767,-122.438904,0.698413,NaN
8,Nob Hill,3050,37.793262,-122.415249,0.617180,NaN
9,Pacific Heights,2973,37.792717,-122.435644,0.689076,NaN


### Second to calculate the RatingScore formula is  (maxgoodrest-currentrestofN)/(maxgoodrest-mingoodrest)

In [20]:
maxgoodrest = max(sf_Neighborhood['GoodRatingRestaurant'])
print('Maximum good restaurant count',maxgoodrest)
mingoodrest = min(sf_Neighborhood['GoodRatingRestaurant'])
print('Minimum good restaurant count',mingoodrest)

rating_Score = pd.DataFrame(columns=['RatingScore'])
for index, Neighborhood in sf_Neighborhood.iterrows():
    currentrentofN = Neighborhood['GoodRatingRestaurant']
    ratingScore =   (maxgoodrest-currentrentofN)/(maxgoodrest-mingoodrest)
    rating_Score = rating_Score.append({'RatingScore' : ratingScore},ignore_index = True)
    
sf_Neighborhood = sf_Neighborhood.join(rating_Score, how='outer')
sf_Neighborhood   



Maximum good restaurant count nan
Minimum good restaurant count nan


,Neighborhood,2018-07,Latitude,Longitude,RentScore,GoodRatingRestaurant,RatingScore
0,Hayes Valley,3030,37.776685,-122.422936,0.635854,NaN,NaN
1,Van Ness - Civic Center,2977,37.775190,-122.419266,0.685341,NaN,NaN
2,Tenderloin,2977,37.784249,-122.413993,0.685341,NaN,NaN
3,Downtown,3040,37.787514,-122.407159,0.626517,NaN,NaN
4,Western Addition,2989,37.779559,-122.429810,0.674136,NaN,NaN
5,Marina,2948,37.799793,-122.435205,0.712418,NaN,NaN
6,Russian Hill,2954,37.797707,-122.414971,0.706816,NaN,NaN
7,Lower Pacific Heights,2963,37.785767,-122.438904,0.698413,NaN,NaN
8,Nob Hill,3050,37.793262,-122.415249,0.617180,NaN,NaN
9,Pacific Heights,2973,37.792717,-122.435644,0.689076,NaN,NaN


### Now we have rent score and rating score, so let's calculate Final Score to decide which are suitable neighborhoods for New Restaurant.

In [21]:
final_Score = pd.DataFrame(columns=['FinalScore'])
for index, Neighborhood in sf_Neighborhood.iterrows():
    rentScore = Neighborhood['RentScore']
    ratingScore = Neighborhood['RatingScore']
    finalScore = (rentScore)*0.6+(ratingScore)*0.4
    final_Score = final_Score.append({'FinalScore' : finalScore},ignore_index = True)
    
sf_Neighborhood = sf_Neighborhood.join(final_Score, how='outer')
sf_Neighborhood

,Neighborhood,2018-07,Latitude,Longitude,RentScore,GoodRatingRestaurant,RatingScore,FinalScore
0,Hayes Valley,3030,37.776685,-122.422936,0.635854,NaN,NaN,NaN
1,Van Ness - Civic Center,2977,37.775190,-122.419266,0.685341,NaN,NaN,NaN
2,Tenderloin,2977,37.784249,-122.413993,0.685341,NaN,NaN,NaN
3,Downtown,3040,37.787514,-122.407159,0.626517,NaN,NaN,NaN
4,Western Addition,2989,37.779559,-122.429810,0.674136,NaN,NaN,NaN
5,Marina,2948,37.799793,-122.435205,0.712418,NaN,NaN,NaN
6,Russian Hill,2954,37.797707,-122.414971,0.706816,NaN,NaN,NaN
7,Lower Pacific Heights,2963,37.785767,-122.438904,0.698413,NaN,NaN,NaN
8,Nob Hill,3050,37.793262,-122.415249,0.617180,NaN,NaN,NaN
9,Pacific Heights,2973,37.792717,-122.435644,0.689076,NaN,NaN,NaN


### Below are the list of suitable neighborhoods with affordable rent in San Francisco for opening a New Restaurant.

In [22]:
Results = sf_Neighborhood[(sf_Neighborhood['FinalScore'] >= 0.8)].sort_values('FinalScore', ascending=False)
Results

,Neighborhood,2018-07,Latitude,Longitude,RentScore,GoodRatingRestaurant,RatingScore,FinalScore


### Folium and choropleth Map of the Best Location for the New Restaurant 